# Traffic Monitoring System

This notebook allows you to run the Traffic Monitoring System on platforms like Google Colab, Kaggle, or any Jupyter environment. The system can:

- **Detect Vehicles in Real-Time**: Spot cars, trucks, and other vehicles in video feeds
- **Read License Plates**: Automatically identify and record license plate numbers
- **Track Moving Vehicles**: Follow each vehicle as it moves through the video
- **Count Traffic**: Count vehicles as they cross a line you define on the screen
- **Store Results**: Save all detection data for later analysis

## How It Works

1. **Video Ingestion**: Gets frames from your video source
2. **Detection**: Uses AI to find vehicles and license plates
3. **Tracking**: Keeps track of each vehicle as it moves
4. **Counting**: Counts vehicles when they cross your defined line
5. **OCR**: Reads license plate text when detected
6. **Storage**: Saves results to a database
7. **Main Application**: Coordinates all the components and shows results

## 1. Setup Environment

First, let's set up the environment and install all necessary dependencies.

In [ ]:
# Check if we're running in Colab or Kaggle
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'kaggle' in sys.modules

print(f"Running in Google Colab: {IN_COLAB}")
print(f"Running in Kaggle: {IN_KAGGLE}")

In [ ]:
# Install necessary packages
!pip install opencv-python-headless numpy ultralytics onnxruntime boxmot easyocr paho-mqtt tqdm

In [ ]:
# Clone the repository if we're in Colab or Kaggle
if IN_COLAB or IN_KAGGLE or not any('traffic_monitoring' in s for s in !ls):
    !git clone https://github.com/leakless21/traffic_monitoring.git
    %cd traffic_monitoring
else:
    print("Repository already exists or we're running locally")

## 2. Download and Prepare Models

Download the AI models needed for vehicle detection and license plate recognition if they don't already exist.

In [ ]:
import os
from pathlib import Path

# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Check if models already exist
vehicle_model_path = Path('./models/yolo11s.onnx')
plate_model_path = Path('./models/plate_v8n.onnx')

if not vehicle_model_path.exists() or not plate_model_path.exists():
    print("Downloading models...")
    !python utils/download_model.py
    print("Models downloaded successfully")
else:
    print("Models already exist, skipping download")

## 3. Upload a Video or Use Sample Data

Now you can either upload your own video or use one of the sample videos.

In [ ]:
# Create a data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# For Colab: provide an upload widget
if IN_COLAB:
    from google.colab import files
    
    print("Please upload a video file:")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        video_path = os.path.join('data', filename)
        with open(video_path, 'wb') as f:
            f.write(uploaded[filename])
        print(f"Uploaded {filename} to {video_path}")
elif IN_KAGGLE:
    # For Kaggle: list available input files
    print("Available input files:")
    !ls ../input
    
    # Symlink input directory to make files accessible
    !ln -s ../input input
else:
    print("Available sample videos:")
    !ls data/*.mp4 data/*.MOV data/*.avi 2>/dev/null || echo "No sample videos found in data/ directory"

## 4. View and Edit Configuration

Let's examine the current configuration and make any necessary changes.

In [ ]:
import yaml

# Display the current configuration
config_path = 'config/settings/config.yaml'

# Ensure the config directory exists
os.makedirs(os.path.dirname(config_path), exist_ok=True)

# Read and display current config if it exists
if os.path.exists(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
        print("Current configuration:")
        print(yaml.dump(config, default_flow_style=False))
else:
    print("No configuration file found. Creating a default configuration.")
    # Create a default configuration
    config = {
        'video': {
            'source': 'data/sample.mp4',  # Will be updated with available video
            'frame_skip': 1,
            'process_resolution': [640, 480],
            'output_fps': 30.0
        },
        'detection': {
            'vehicle_model': 'models/yolo11s.onnx',
            'plate_model': 'models/plate_v8n.onnx',
            'vehicle_model_version': 'yolo11',
            'plate_model_version': 'yolov8',
            'confidence': 0.25,
            'iou_threshold': 0.45
        },
        'tracking': {
            'tracker_type': 'bytetrack',
            'confidence': 0.3
        },
        'counting': {
            'use_relative_coordinates': True,
            'line': {
                'start': [0.25, 0.6],
                'end': [0.75, 0.6]
            },
            'raw_coordinates': {
                'start': [320, 360],
                'end': [960, 360]
            }
        },
        'ocr': {
            'languages': ['en'],
            'use_gpu': False
        },
        'hardware': {
            'use_gpu': False,
            'provider': 'auto',
            'precision': 'fp32'
        },
        'storage': {
            'database_path': 'data/traffic_data.db',
            'save_images': False
        },
        'mqtt': {
            'enabled': False,
            'broker': 'localhost',
            'port': 1883,
            'topic_prefix': 'traffic'
        }
    }

    with open(config_path, 'w') as file:
        yaml.dump(config, file, default_flow_style=False)
    print("Default configuration created.")

## 5. Select Video Source

Choose which video to process and update the configuration.

In [ ]:
# List available videos
import glob

# Search for videos in different locations
all_videos = []
for ext in ['*.mp4', '*.avi', '*.MOV', '*.mkv']:
    # Look in data directory
    all_videos.extend(glob.glob(os.path.join('data', ext)))
    # For Kaggle, look in input directory if exists
    if IN_KAGGLE and os.path.exists('input'):
        all_videos.extend(glob.glob(os.path.join('input', ext)))
        all_videos.extend(glob.glob(os.path.join('input', '*', ext)))

print("Found videos:")
for i, video in enumerate(all_videos):
    print(f"{i+1}. {video}")

# Allow selection
if all_videos:
    selection = input(f"Enter the number of the video to use (1-{len(all_videos)}): ")
    try:
        idx = int(selection) - 1
        if 0 <= idx < len(all_videos):
            selected_video = all_videos[idx]
            # Update configuration with selected video
            with open(config_path, 'r') as file:
                config = yaml.safe_load(file)
            config['video']['source'] = selected_video
            with open(config_path, 'w') as file:
                yaml.dump(config, file, default_flow_style=False)
            print(f"Configuration updated to use {selected_video}")
        else:
            print(f"Invalid selection. Please run the cell again.")
    except ValueError:
        print(f"Invalid input. Please run the cell again.")
else:
    print("No videos found. Please upload a video first.")

## 6. Edit Configuration (Optional)

You can manually adjust various settings to customize how the system works. Add, remove, or modify entries as needed.

In [ ]:
# Load current config
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Here you can modify any config parameters as needed
# For example:
config['video']['frame_skip'] = 1  # Process every frame (2 would process every other frame)
config['detection']['confidence'] = 0.25  # Lower = more detections but more false positives
config['hardware']['use_gpu'] = False  # Set to True if your system has GPU support

# Adjust counting line position (vertical position from 0-1)
count_line_position = 0.6
config['counting']['line']['start'][1] = count_line_position
config['counting']['line']['end'][1] = count_line_position

# Save the updated config
with open(config_path, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

print("Configuration updated successfully.")

## 7. Run Traffic Monitoring System

Now let's run the traffic monitoring system using the main.py script.

In [ ]:
import subprocess
import time
import os
from IPython.display import HTML, display
from pathlib import Path

# Get output directory path for recordings
output_dir = Path('data/recordings')
output_dir.mkdir(parents=True, exist_ok=True)

# Generate a timestamp for the output file
timestamp = time.strftime("%Y%m%d_%H%M%S")
output_file = str(output_dir / f"traffic_monitoring_{timestamp}.mp4")

print(f"Will save output to: {output_file}")

# Run main.py with arguments
if IN_COLAB or IN_KAGGLE:
    # In Colab/Kaggle, we need to run in headless mode (no UI) and just record
    cmd = [sys.executable, 'main.py', '--no-ui', '--record', f'--output={output_file}']
else:
    # If running locally, show UI and also record
    cmd = [sys.executable, 'main.py', '--record', f'--output={output_file}']

print(f"Running command: {' '.join(cmd)}")

# Execute the command
try:
    # Run the process and capture output
    process = subprocess.Popen(
        cmd, 
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    # Print output in real-time
    print("Process started. Output:")
    for line in process.stdout:
        print(line, end='')
        
    # Wait for process to complete
    process.wait()
    print(f"Process completed with return code: {process.returncode}")
except KeyboardInterrupt:
    print("\nProcess interrupted by user.")
    # Try to terminate the process gracefully
    process.terminate()
    try:
        process.wait(timeout=5)
    except subprocess.TimeoutExpired:
        process.kill()
except Exception as e:
    print(f"Error running traffic monitoring: {e}")

## 8. Display Results

Let's display the output video and provide download options.

In [ ]:
from IPython.display import HTML, display
from base64 import b64encode

# Function to create HTML video player for notebook
def display_video(video_path):
    if not os.path.exists(video_path):
        print(f"Video file not found: {video_path}")
        return
    
    # Get video data
    video_data = open(video_path, 'rb').read()
    
    # Encode as base64
    video_base64 = b64encode(video_data).decode('utf-8')
    
    # Create HTML with video player
    html = f'''
    <div style="display: flex; flex-direction: column; align-items: center;">
        <h3>Processed Video: {os.path.basename(video_path)}</h3>
        <video width="640" height="480" controls>
            <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
            Your browser does not support the video tag.
        </video>
    </div>
    '''
    
    # Display the video
    display(HTML(html))

# Find and display the most recent output video
output_videos = glob.glob('data/recordings/*.mp4')

if output_videos:
    # Sort by modification time to get the most recent
    latest_video = max(output_videos, key=os.path.getmtime)
    print(f"Displaying most recent output video: {latest_video}")
    display_video(latest_video)
    
    # Provide download link for Colab users
    if IN_COLAB:
        from google.colab import files
        print("\nDownload the processed video:")
        files.download(latest_video)
else:
    print("No output videos found. The processing may have failed or been interrupted.")

## 9. Analyze Results from Database

Optionally, let's explore the data collected in the SQLite database.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to the database
db_path = 'data/traffic_data.db'

if os.path.exists(db_path):
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        
        # Get table names
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        print("Database tables:")
        for table in tables:
            print(f"- {table[0]}")
            
        # Query vehicle counts
        df_counts = pd.read_sql("SELECT * FROM vehicle_counts ORDER BY timestamp DESC LIMIT 100", conn)
        if not df_counts.empty:
            print(f"\nVehicle counts (most recent {len(df_counts)} records):")
            display(df_counts.head())
            
            # Plot counts over time if data exists
            if len(df_counts) > 1:
                plt.figure(figsize=(10, 6))
                plt.plot(df_counts['timestamp'], df_counts['count'], marker='o')
                plt.title('Vehicle Counts Over Time')
                plt.xlabel('Timestamp')
                plt.ylabel('Count')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
        
        # Query license plates
        df_plates = pd.read_sql("SELECT * FROM license_plates ORDER BY timestamp DESC LIMIT 100", conn)
        if not df_plates.empty:
            print(f"\nDetected license plates (most recent {len(df_plates)} records):")
            display(df_plates.head())
            
        # Close connection
        conn.close()
        
    except Exception as e:
        print(f"Error accessing database: {e}")
else:
    print(f"Database file not found: {db_path}")

## 10. Conclusion

You've successfully run the Traffic Monitoring System! Here's what we accomplished:

1. Set up the necessary environment
2. Downloaded the required AI models
3. Configured the system by directly updating the config.yaml file
4. Processed a video to detect, track, count vehicles and read license plates
5. Saved and analyzed the results

This approach is more efficient as it leverages the existing main.py infrastructure rather than reimplementing functionality in the notebook.

### Next Steps

- Try different videos and configurations
- Explore the database in more detail for advanced analytics
- If you're interested in the technical details, explore the source code in the repository